In [23]:
import numpy as np
import pandas as pd 
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import nltk 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import requests
from datetime import datetime

import yfinance as yf

In [67]:
data=yf.download(tickers='ZM', period='max', interval='1d')
data['next day adj close'] = data['Adj Close'].shift(-1)
data.reset_index(inplace=True)

[*********************100%***********************]  1 of 1 completed


In [29]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,next day adj close
0,2019-04-18,65.000000,66.000000,60.320999,62.000000,62.000000,25764700,65.699997
1,2019-04-22,61.000000,68.900002,59.939999,65.699997,65.699997,9949700,69.000000
2,2019-04-23,66.870003,74.168999,65.550003,69.000000,69.000000,6786500,63.200001
3,2019-04-24,71.400002,71.500000,63.160000,63.200001,63.200001,4973500,65.000000
4,2019-04-25,64.739998,66.849998,62.599998,65.000000,65.000000,3863300,66.220001


# company news
summary is not full

should be grouped by date and mean sentiment

In [75]:
def get_news(company, date_from='2021-06-01', date_to=None):
    '''
    returns dataframe with average sentiment of news headline and sentiment of news summary for every date in a given timeframe
    company: symbol, example ZM
    date_from: string format yyyy-mm-dd
    date_to: string format yyyy-mm-dd
    '''
    sid_obj = SentimentIntensityAnalyzer()
    if date_to is None:
        date_to = datetime.today().strftime("%Y-%m-%d")
    URL = 'https://finnhub.io/api/v1/company-news?symbol={}&from={}&to={}&token=c2si65iad3ic1qis06lg'.format(company, date_from, date_to)
    r = requests.get(URL)
    news_df = pd.DataFrame(r.json())
    news_df['datetime'] = [datetime.utcfromtimestamp(i).strftime('%Y-%m-%d') for i in news_df.datetime]
    news_df.drop(['id','image', 'related','source', 'url'], axis=1, inplace=True)
    news_df['headline_sentiment'] = [sid_obj.polarity_scores(c)['compound'] for c in news_df['headline']]
    news_df['summary_sentiment'] = [sid_obj.polarity_scores(c)['compound'] for c in news_df['summary']]
    
    news_dates = news_df.groupby(['datetime']).mean().sort_index().reset_index()

    news_dates['Date'] = pd.to_datetime(news_dates['datetime'], format='%Y-%m-%d')
    return news_dates

In [76]:
news_df = get_news('ZM', '2021-01-01')
news_df

,datetime,headline_sentiment,summary_sentiment,Date
0,2020-05-26,0.202300,0.848100,2020-05-26
1,2021-04-19,-0.022191,0.046955,2021-04-19
2,2021-04-20,0.225430,0.423340,2021-04-20
3,2021-04-21,0.139536,0.305936,2021-04-21
4,2021-04-22,0.092689,0.255689,2021-04-22
...,...,...,...,...
57,2021-06-18,0.460917,0.315800,2021-06-18
58,2021-06-19,0.391067,0.689467,2021-06-19
59,2021-06-20,0.000000,0.000000,2021-06-20
60,2021-06-21,0.090050,0.343317,2021-06-21


In [77]:
news_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   datetime            62 non-null     object        
 1   headline_sentiment  62 non-null     float64       
 2   summary_sentiment   62 non-null     float64       
 3   Date                62 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 2.1+ KB


In [69]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549 entries, 0 to 548
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                549 non-null    datetime64[ns]
 1   Open                549 non-null    float64       
 2   High                549 non-null    float64       
 3   Low                 549 non-null    float64       
 4   Close               549 non-null    float64       
 5   Adj Close           549 non-null    float64       
 6   Volume              549 non-null    int64         
 7   next day adj close  548 non-null    float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 34.4 KB


# Build stock + sentiment news

In [71]:
data = data.merge(news_df, how='inner', on='Date')#.drop(['Date','datetime'],1)
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,next day adj close,datetime,headline_sentiment,summary_sentiment
0,2020-05-26,169.100006,169.690002,163.119995,164.000000,164.000000,6689700,161.970001,2020-05-26,0.202300,0.848100
1,2021-04-19,327.519989,337.795013,320.500000,324.329987,324.329987,3131900,321.779999,2021-04-19,-0.022191,0.046955
2,2021-04-20,326.730011,327.549988,316.359985,321.779999,321.779999,2282500,322.760010,2021-04-20,0.225430,0.423340
3,2021-04-21,317.010010,323.109985,313.500000,322.760010,322.760010,1917500,328.320007,2021-04-21,0.139536,0.305936
4,2021-04-22,322.665009,338.049988,322.665009,328.320007,328.320007,3328300,336.880005,2021-04-22,0.092689,0.255689


# News Sentiment
Get company's news sentiment and statistics. This endpoint is only available for US companies.

get only one raw for the last week

buzz
Statistics of company news in the past week.

articlesInLastWeek
buzz
weeklyAverage
companyNewsScore
News score.

sectorAverageBullishPercent
Sector average bullish percent.

sectorAverageNewsScore
Sectore average score.

sentiment
News sentiment.

bearishPercent
bullishPercent
symbol
Requested symbol.

In [11]:
r = requests.get('https://finnhub.io/api/v1/news-sentiment?symbol=ZM&token=c2si65iad3ic1qis06lg')
print(r.json())

{'buzz': {'articlesInLastWeek': 50, 'buzz': 0.9708, 'weeklyAverage': 51.5}, 'companyNewsScore': 0.6739, 'sectorAverageBullishPercent': 0.6643, 'sectorAverageNewsScore': 0.5393, 'sentiment': {'bearishPercent': 0, 'bullishPercent': 1}, 'symbol': 'ZM'}


# Basic Financials
Get company basic financials such as margin, P/E ratio, 52-week high/low etc.

quarterly /  annually

In [20]:
r = requests.get('https://finnhub.io/api/v1/stock/metric?symbol=ZM&metric=all&token=c2si65iad3ic1qis06lg')
print(r.json()['metric'])
print(r.json()['series']['annual'])

{'10DayAverageTradingVolume': 4.40122, '13WeekPriceReturnDaily': 5.77673, '26WeekPriceReturnDaily': -7.71013, '3MonthAverageTradingVolume': 69.97736, '52WeekHigh': 588.84, '52WeekHighDate': '2020-10-19', '52WeekLow': 216.75, '52WeekLowDate': '2020-06-12', '52WeekPriceReturnDaily': 64.99302, '5DayPriceReturnDaily': 9.04762, 'assetTurnoverAnnual': 0.80493, 'assetTurnoverTTM': 0.82436, 'beta': None, 'bookValuePerShareAnnual': 13.15203, 'bookValuePerShareQuarterly': 14.23621, 'bookValueShareGrowth5Y': None, 'capitalSpendingGrowth5Y': None, 'cashFlowPerShareAnnual': 2.35192, 'cashFlowPerShareTTM': 3.01615, 'cashPerSharePerShareAnnual': 14.45997, 'cashPerSharePerShareQuarterly': 15.92322, 'currentDividendYieldTTM': 0, 'currentEv/freeCashFlowAnnual': 168.5417, 'currentEv/freeCashFlowTTM': 137.7837, 'currentRatioAnnual': 3.80396, 'currentRatioQuarterly': 3.49458, 'dividendGrowthRate5Y': None, 'dividendPerShare5Y': None, 'dividendPerShareAnnual': None, 'dividendYield5Y': None, 'dividendYieldInd

# Recommendation Trends
Get latest analyst recommendation trends for a company.

per month

symbol
Company symbol.

buy
Number of recommendations that fall into the Buy category

hold
Number of recommendations that fall into the Hold category

period
Updated period

sell
Number of recommendations that fall into the Sell category

strongBuy
Number of recommendations that fall into the Strong Buy category

strongSell
Number of recommendations that fall into the Strong Sell category


In [84]:
def get_recommendation_trends(company):
    r = requests.get('https://finnhub.io/api/v1/stock/recommendation?symbol=ZM&token=c2si65iad3ic1qis06lg')
    recommendation_df = pd.DataFrame(r.json())
    dfs = []
    for i, month in recommendation_df.period.iteritems():
        period = month
        df_month = pd.DataFrame({
        'period': pd.date_range(
            start = pd.Timestamp(period),                        
            end = pd.Timestamp(period) + pd.offsets.MonthEnd(0),  # <-- 2018-08-31 with MonthEnd
            freq = 'D'
            )
        })
        df_month['month'] = month
        dfs.append(df_month)
    df = pd.concat(dfs)
    df.period = [datetime.strftime(x, '%Y-%m-%d') for x in df.period]
    df = df.merge(recommendation_df,how='left', left_on='month', right_on='period')
    df.drop(['month','period_y'], axis=1, inplace=True)
    df['Date'] = pd.to_datetime(df['period_x'], format='%Y-%m-%d')
    return df.drop(['period_x'],1)

In [85]:
trend_df = get_recommendation_trends('ZM')
trend_df

,buy,hold,sell,strongBuy,strongSell,symbol,Date
0,9,14,0,10,1,ZM,2021-06-01
1,9,14,0,10,1,ZM,2021-06-02
2,9,14,0,10,1,ZM,2021-06-03
3,9,14,0,10,1,ZM,2021-06-04
4,9,14,0,10,1,ZM,2021-06-05
...,...,...,...,...,...,...,...
787,0,3,4,1,3,ZM,2019-05-27
788,0,3,4,1,3,ZM,2019-05-28
789,0,3,4,1,3,ZM,2019-05-29
790,0,3,4,1,3,ZM,2019-05-30


In [86]:
trend_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 792 entries, 0 to 791
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   buy         792 non-null    int64         
 1   hold        792 non-null    int64         
 2   sell        792 non-null    int64         
 3   strongBuy   792 non-null    int64         
 4   strongSell  792 non-null    int64         
 5   symbol      792 non-null    object        
 6   Date        792 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 49.5+ KB


# build stock + news sentiment + recommendation

In [87]:
data = data.merge(trend_df, how='inner', on='Date')#.drop(['Date','datetime'],1)
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,next day adj close,datetime,headline_sentiment,summary_sentiment,buy,hold,sell,strongBuy,strongSell,symbol
0,2020-05-26,169.100006,169.690002,163.119995,164.000000,164.000000,6689700,161.970001,2020-05-26,0.202300,0.848100,8,15,4,8,0,ZM
1,2021-04-19,327.519989,337.795013,320.500000,324.329987,324.329987,3131900,321.779999,2021-04-19,-0.022191,0.046955,12,14,1,9,1,ZM
2,2021-04-20,326.730011,327.549988,316.359985,321.779999,321.779999,2282500,322.760010,2021-04-20,0.225430,0.423340,12,14,1,9,1,ZM
3,2021-04-21,317.010010,323.109985,313.500000,322.760010,322.760010,1917500,328.320007,2021-04-21,0.139536,0.305936,12,14,1,9,1,ZM
4,2021-04-22,322.665009,338.049988,322.665009,328.320007,328.320007,3328300,336.880005,2021-04-22,0.092689,0.255689,12,14,1,9,1,ZM


# Social Sentiment
Get social sentiment for stocks on Reddit and Twitter. This endpoint is currently in Beta.

several per day

In [37]:
r = requests.get('https://finnhub.io/api/v1/stock/social-sentiment?symbol=ZM&token=c2si65iad3ic1qis06lg&from=2021-06-01')
print(r.json()['reddit'][0:10])
reddit_df = pd.DataFrame(r.json()['reddit'])
reddit_df.head()

[{'atTime': '2021-06-13 22:00:00', 'mention': 1, 'positiveScore': 0, 'negativeScore': 0.980246, 'positiveMention': 0, 'negativeMention': 1, 'score': -0.980246}, {'atTime': '2021-06-13 00:00:00', 'mention': 26, 'positiveScore': 0, 'negativeScore': 0, 'positiveMention': 0, 'negativeMention': 0, 'score': 0}, {'atTime': '2021-06-12 23:00:00', 'mention': 86, 'positiveScore': 0, 'negativeScore': 0, 'positiveMention': 0, 'negativeMention': 0, 'score': 0}, {'atTime': '2021-06-12 22:00:00', 'mention': 29, 'positiveScore': 0, 'negativeScore': 0, 'positiveMention': 0, 'negativeMention': 0, 'score': 0}, {'atTime': '2021-06-12 21:00:00', 'mention': 17, 'positiveScore': 0, 'negativeScore': 0, 'positiveMention': 0, 'negativeMention': 0, 'score': 0}, {'atTime': '2021-06-11 21:00:00', 'mention': 1, 'positiveScore': 0.9996642, 'negativeScore': 0, 'positiveMention': 1, 'negativeMention': 0, 'score': 0.9996642}, {'atTime': '2021-06-11 18:00:00', 'mention': 1, 'positiveScore': 0, 'negativeScore': 0.9149015

,atTime,mention,positiveScore,negativeScore,positiveMention,negativeMention,score
0,2021-06-13 22:00:00,1,0.0,0.980246,0,1,-0.980246
1,2021-06-13 00:00:00,26,0.0,0.000000,0,0,0.000000
2,2021-06-12 23:00:00,86,0.0,0.000000,0,0,0.000000
3,2021-06-12 22:00:00,29,0.0,0.000000,0,0,0.000000
4,2021-06-12 21:00:00,17,0.0,0.000000,0,0,0.000000


# list of companies

8124

description
Symbol description

displaySymbol
Display symbol name.

symbol
Unique symbol used to identify this symbol used in /stock/candle endpoint.

type
Security type.


In [35]:
r = requests.get('https://finnhub.io/api/v1/search?token=c2si65iad3ic1qis06lg')
print(r.json()['result'][0:10])
print(len(r.json()['result']))

[{'description': 'Forex.com AUD/JPY', 'displaySymbol': 'AUD/JPY', 'symbol': 'FOREX:401484303', 'type': 'FX'}, {'description': 'ZB QUN/BTC', 'displaySymbol': 'QUN/BTC', 'symbol': 'ZB:qun_btc', 'type': 'Crypto'}, {'description': 'Binance NMR/USDT', 'displaySymbol': 'NMR/USDT', 'symbol': 'BINANCE:NMRUSDT', 'type': 'Crypto'}, {'description': 'Kucoin LYM/USDT', 'displaySymbol': 'LYM/USDT', 'symbol': 'KUCOIN:LYM-USDT', 'type': 'Crypto'}, {'description': 'BITFINEX GOT/USD', 'displaySymbol': 'GOT/USD', 'symbol': 'BITFINEX:GOTUSD', 'type': 'Crypto'}, {'description': 'Huobi SALT/BTC', 'displaySymbol': 'SALT/BTC', 'symbol': 'HUOBI:saltbtc', 'type': 'Crypto'}, {'description': 'FXPIG UKOIL UKOIL', 'displaySymbol': 'UKOIL352/USD', 'symbol': 'FXPIG:352', 'type': 'Oil'}, {'description': 'BITFINEX ETP/ETH', 'displaySymbol': 'ETP/ETH', 'symbol': 'BITFINEX:ETPETH', 'type': 'Crypto'}, {'description': 'Binance TRX/PAX', 'displaySymbol': 'TRX/PAX', 'symbol': 'BINANCE:TRXPAX', 'type': 'Crypto'}, {'descriptio